In [1]:
import scanpy as sc
import pandas as pd
from pathlib import Path
import anndata as ad
import numpy as np
import os
import scvi

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

DPI = 300
FONTSIZE = 20  # 42

sc.settings.set_figure_params(
    scanpy=True, dpi=100, transparent=True, vector_friendly=True, dpi_save=DPI
)
from matplotlib import rcParams

rcParams["pdf.fonttype"] = 42

Global seed set to 0


In [2]:
DIR2SAVE = Path(
    "/data/BCI-CRC/nasrine/data/CRC/Metastatic_CRC_LM_dataset/subpopulations/TNKILC/latent20"
)
DIR2SAVE.mkdir(parents=True, exist_ok=True)

In [3]:
FIG2SAVE = DIR2SAVE.joinpath("figures/")
FIG2SAVE.mkdir(parents=True, exist_ok=True)
# set the global variable: sc.settings.figdir to save all plots
sc.settings.figdir = FIG2SAVE

In [4]:
adata = sc.read_h5ad(
    DIR2SAVE.joinpath("Multiome_Che_Wu_CRC_LM_integrated_scvi_hvg_TNKILC.h5ad")
)

In [5]:
adata

AnnData object with n_obs × n_vars = 82184 × 2000
    obs: 'Patient', 'Sample', 'Cell_type', 'Cell_subtype', 'Tissue', 'Therapy', 'doublet_score', 'n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo', 'cell_source', 'Annotation_scVI', 'S_score', 'G2M_score', 'phase', 'cell_cycle_diff', '_scvi_batch', '_scvi_labels'
    var: 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'Annotation_scVI_colors', 'Patient_colors', 'Therapy_colors', 'Tissue_colors', '_scvi_manager_uuid', '_scvi_uuid', 'cell_source_colors', 'hvg', 'log1p', 'neighbors', 'pca', 'phase_colors', 'umap'
    obsm: 'X_pca', 'X_scVI', 'X_umap', '_scvi_extra_categorical_covs', '_scvi_extra_continuous_covs'
    varm: 'PCs'
    layers: 'log1p', 'normalised', 'raw', 'scvi_normalized'
    obsp: 'connectivities', 'distances'

In [6]:
adata.obs.cell_source.value_counts()

Wu-Cells      41894
Che-Cells     38645
BCI-Nuclei     1645
Name: cell_source, dtype: int64

### Leiden clustering

In [ ]:
adata.uns["neighbors"]

In [ ]:
from collections import Counter


def clustering_leiden_resolution(adata2test, res_range):
    """
    Performs hyperparameter search for resolution in leiden clustering
    :param adata2test: AnnData frame
    :param res_range: range of values to evaluate: i.e. np.arange(0.1, 1.5, 0.05)
    """
    resolution_dict = {r: None for r in res_range}
    # vary resolution parameter and see which nb of clusters occurs more frequently
    for r in res_range:
        # load adata
        adata = adata2test.copy()
        sc.tl.leiden(adata, resolution=r, random_state=7)
        # store nb of clusters for that resolution
        resolution_dict[r] = len(set(adata.obs["leiden"]))

    # plot figure: nb clusters in fct of resolution param
    fig, ax = plt.subplots(
        nrows=1, ncols=1, sharey=False, sharex=False, dpi=DPI, figsize=(5, 4.5)
    )
    plt.scatter(resolution_dict.keys(), resolution_dict.values())
    ax.set_xlabel("Resolution")
    ax.set_ylabel("Number of clusters")
    ax.tick_params(axis="both", which="major", labelsize=FONTSIZE - 10)
    plt.show()

    # display nb of times each number of clusters occurs
    print("Frequency of NB clusters")
    print(Counter(list(resolution_dict.values())))

In [ ]:
clustering_leiden_resolution(adata, res_range=np.arange(0.1, 1.5, 0.1))

In [ ]:
adata.obs.cell_source.value_counts()

In [ ]:
# neighbors were already computed using Harmony corrected PCs, so we perform clustering on that graph
sc.tl.leiden(adata, key_added="leiden_scVI", resolution=1.3, random_state=7)

In [ ]:
sc.pl.umap(
    adata,
    color="leiden_scVI",
    legend_loc="on data",
    save="general_clustering.pdf",
    show=True,
)

In [ ]:
sc.tl.leiden(adata, key_added="leiden_scVI_r1", resolution=1, random_state=7)
sc.pl.umap(
    adata,
    color="leiden_scVI_r1",
    legend_loc="on data",
    save="general_clustering_r1.pdf",
    show=True,
)

In [ ]:
sc.tl.leiden(adata, key_added="leiden_scVI_r0.4", resolution=0.4, random_state=7)
sc.pl.umap(
    adata,
    color="leiden_scVI_r0.4",
    legend_loc="on data",
    save="general_clustering_r0.4.pdf",
    show=True,
)

In [ ]:
sc.pl.umap(
    adata,
    color=["GZMB", "GZMK", "CD8A", "CD8B"],
    show=True,
)

In [ ]:
adata.obs["leiden_scVI"].value_counts()

In [ ]:
adata.obs.Patient.value_counts()

In [ ]:
sc.pl.umap(
    adata,
    color=["S_score", "G2M_score", "phase", "cell_cycle_diff"],
    color_map="viridis",
    save="cell_cycle.pdf",
)

In [ ]:
sc.pl.umap(
    adata,
    color=[
        "doublet_score",
        "n_genes_by_counts",
        "pct_counts_mt",
        "pct_counts_ribo",
    ],
    color_map="viridis",
    save="QC_covariates.pdf",
)

### Look at distribution of cell source across clusters 

In [ ]:
from matplotlib.patches import Rectangle


def proportion_cells_patient(
    adata, groupby_labels, xlabel: str, ylabel: str, colors: dict, figname: str
):  # colors
    # compute proportion of cells within each group
    table2plot = (
        adata.reset_index()
        .groupby(groupby_labels)
        .size()
        .groupby(level=0)
        .apply(lambda x: x * 100 / x.sum())
        .unstack()
    )

    fig, ax = plt.subplots(
        nrows=1,
        ncols=1,
        sharey=False,
        sharex=False,
        dpi=DPI,
    )  # figsize=(4, 4))

    print(table2plot)
    table2plot.plot.bar(stacked=True, ax=ax, color=colors.values())  # , color=colors
    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)
    ax.grid(False, which="major", axis="both")  # removes major horizontal gridlinesd

    labels = list(colors.keys())
    l = [Rectangle((0, 0), 0, 0, color=color) for color in list(colors.values())]
    ax.legend(
        l,
        labels,
        loc="upper left",
        bbox_to_anchor=(1, 0.8),
        facecolor="white",
        edgecolor="white",
        ncol=1,
        borderaxespad=0.0,
        framealpha=0,
        frameon=False,
    )

    plt.savefig(FIG2SAVE.joinpath(figname), dpi=DPI, format="pdf", bbox_inches="tight")
    plt.show()

In [ ]:
from collections import OrderedDict

color_dict = OrderedDict(
    zip(
        adata.obs["cell_source"].cat.categories.values.tolist(),
        adata.uns["cell_source_colors"],
    )
)

proportion_cells_patient(
    adata.obs,
    groupby_labels=["leiden_scVI", "cell_source"],
    xlabel="leiden_scVI",
    ylabel="Percent Cell source",
    colors=color_dict,
    figname="leiden_cell_source.pdf",
)

### Plot some markers 

In [ ]:
markers_T_general = {
    "T": ["TRAC"],
    "CD4": ["CD4"],
    "CD8": ["CD8A", "CD8B"],
    "Cycling": ["MKI67"],
    "Treg": ["FOXP3", "CTLA4", "CCR4", "IL2RA"],  # Treg are CD127 (IL7R) low.
    "Exhausted T": [
        "PDCD1",
        "LAG3",
        "HAVCR2",
        "CTLA4",
        "TIGIT",
        "ENTPD1",
    ],  # PD1: PCDC1, TIM3: HAVCR2,  CD39:ENTPD1 https://www.nature.com/articles/s41467-021-23324-4
    "NK": [
        "EOMES",
        "CMC1",
        "GZMK",
        "XCL1",
        "NKG7",
        "PRF1",
        "NCR1",
        "KLRC1",
        "NCAM1",
        "FCER1G",
        "ITGA1",
        "GZMB",
        "FCGR3A",
    ],
    "ILC": ["AREG", "TLE1", "IL4I1"],
}

markers_T_cd4 = {
    "Th": ["CD4", "IL7R", "CD40LG", "ANXA1"],
    "Tfh": ["ITM2A", "LPAR6", "PDCD1"],
    "Naïve CD4 T": [
        "CCR7",
        "SELL",
        "TCF7",
        "LEF1",
    ],  # https://www.nature.com/articles/s41467-019-12464-3
    "Th1/Th17/Th2": [
        "CXCR3",
        "TBX21",
        "CCL5",
        "CCR6",
        "IL22",
        "RORA",
        "IL7R",
        "IL4",
        "IL13",
        "GATA3",
        "CCR4",
    ],
    "Th17": [
        "IL17A",
        "ODF2L",
        "IL7R",
        "PDE4D",
    ],  #'CCR4', 'CCR6', 'IL1R1', 'IL6R', 'IL21R', 'IL23R'],# 'TGFBR1', 'RORA', 'RORC', 'BATF', 'IRF4'], # IL17 not in data
    "Th1": ["CCL5", "PHLDA1", "LYAR"],
}

markers_T_cd8 = {
    "Effector CD8": [
        "CCL4",
        "CCL5",
        "GZMK",
        "GZMB",
        "PFN1",
        "GZMA",
        "GZMH",
        "NKG7",
    ],  # Cytotoxic is same as effector for the effector, if it’s only one cluster, and doesn’t have TCF7/CCR7, I would annotate them just as effector
    "Tmem": [
        "CCR7",
        "PTPRC",
        "ENPP1",
    ],  # https://panglaodb.se/markers.html?cell_type=%27T%20memory%20cells%27
    "Naive cytotoxic": ["CD8A", "CCR7", "SELL"],
}

markers_T_other = {
    "gdT": ["KLRC2", "TRGC1", "TRGC2", "TRDC"],  # not in data: 'TCRD','TCRG'
    "NKT": ["GZMA", "CCL5", "NKG7", "KLRB1", "CD3G", "FGFBP2"],
    "MAIT": ["SLC4A10", "NCR3", "KLRB1"],
}

markers_ILC = {
    "ILC": [
        "AREG",
        "TLE1",
        "IL4I1",
    ],  # ['NCR2', 'ITGAE', 'KIT', 'IL7R', 'KLRB1', 'AHR'],
    "ILC1": ["TBX21", "CD3D", "CXCR3", "PLCD4"],
    "ILC2": ["KRT1", "HPGDS", "SLAMF1"],  # ['HPGDS', 'GATA3', 'PTGDR2', 'IL1RL1'],
    "ILC3": ["IL4I1", "RORC", "TNFRSF25", "SPINK2", "KLRB1", "IL7R"],
}

In [ ]:
import itertools

# use log1p data stored in .raw
markers2plot = list(
    itertools.chain(*list(markers_T_general.values()))
)  # get all markers in a single list
sc.pl.umap(
    adata,
    color=markers2plot,
    use_raw=True,
    vmin=0.0,
    vmax="p99",
    color_map="plasma_r",  #'RdPu',
    save="general_markers.pdf",
    show=True,
)

In [ ]:
import itertools

# use log1p data stored in .raw
markers2plot = list(
    itertools.chain(*list(markers_T_cd4.values()))
)  # get all markers in a single list
sc.pl.umap(
    adata,
    color=markers2plot,
    use_raw=True,
    vmin=0.0,
    vmax="p99",
    color_map="plasma_r",  #'RdPu',
    save="markers_cd4.pdf",
    show=True,
)

In [ ]:
import itertools

# use log1p data stored in .raw
markers2plot = list(
    itertools.chain(*list(markers_T_cd8.values()))
)  # get all markers in a single list
sc.pl.umap(
    adata,
    color=markers2plot,
    use_raw=True,
    vmin=0.0,
    vmax="p99",
    color_map="plasma_r",  #'RdPu',
    save="markers_cd8.pdf",
    show=True,
)

In [ ]:
import itertools

# use log1p data stored in .raw
markers2plot = list(
    itertools.chain(*list(markers_T_other.values()))
)  # get all markers in a single list
sc.pl.umap(
    adata,
    color=markers2plot,
    use_raw=True,
    vmin=0.0,
    vmax="p99",
    color_map="plasma_r",  #'RdPu',
    save="markers_other.pdf",
    show=True,
)

In [ ]:
import itertools

# use log1p data stored in .raw
markers2plot = list(
    itertools.chain(*list(markers_ILC.values()))
)  # get all markers in a single list
sc.pl.umap(
    adata,
    color=markers2plot,
    use_raw=True,
    vmin=0.0,
    vmax="p99",
    color_map="plasma_r",  #'RdPu',
    save="markers_ILC.pdf",
    show=True,
)

In [ ]:
### markers for stress
# HSP
dissocation_markers_dict = {
    "shock protein": [
        "HSPE1",
        "HSPA1A",
        "HSPA1B",
        "HSP90AA1",
        "HSP90AB1",
        "HSPA8",
        "HSPB1",
    ],
    "immediate early genes": ["FOS", "JUN"],
}

sc.pl.umap(
    adata,
    color=dissocation_markers_dict["shock protein"],
    vmax="p99",
    use_raw=True,
    vmin=0,
    color_map="plasma_r",
    save="general_HSPmarkers.pdf",
    show=True,
)

In [ ]:
sc.pl.umap(adata, color="Cell_subtype")

In [ ]:
ax = sc.pl.umap(adata, size=10, show=False)
sc.pl.umap(
    adata[adata.obs["Cell_subtype"] == "Treg"], size=10, color="Cell_subtype", ax=ax
)

In [ ]:
ax = sc.pl.umap(adata, size=10, show=False)
sc.pl.umap(
    adata[adata.obs["cell_source"] == "Wu-Cells"], size=10, color="Cell_subtype", ax=ax
)

In [ ]:
adata.obs[["leiden_scVI", "cell_source"]].groupby(["leiden_scVI", "cell_source"]).size()

### Differential expression to get DE genes upregulated per cluster 

In [ ]:
# issue here https://github.com/theislab/single-cell-tutorial/issues/97
# This seems to be a scanpy bug as you can see here and here. The latter issue suggests to just add the line:
# adata.uns['log1p']["base"] = None after reading again, or downgrading to AnnData<0.8.
# Either way, this should be fixed soon by the maintenance team.
adata.uns["log1p"]["base"] = None

In [ ]:
sc.tl.rank_genes_groups(
    adata,
    groupby="leiden_scVI",
    reference="rest",
    method="wilcoxon",
    use_raw=True,
    layer=None,
    pts=True,
    corr_method="benjamini-hochberg",
    key_added="rank_genes_wilcoxon",
)

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata,
    groupby="leiden_scVI",
    key="rank_genes_wilcoxon",
    var_names=markers_T_general,
    values_to_plot="logfoldchanges",
    cmap="bwr",
    vmin=-4,
    vmax=4,
    min_logfoldchange=1,
    colorbar_title="log fold change",
    save="general_dotplot.pdf",
    show=True,
)

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata,
    groupby="leiden_scVI",
    key="rank_genes_wilcoxon",
    var_names=markers_T_cd4,
    values_to_plot="logfoldchanges",
    cmap="bwr",
    vmin=-4,
    vmax=4,
    min_logfoldchange=1,
    colorbar_title="log fold change",
    save="general_dotplot_cd4.pdf",
    show=True,
)

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata,
    groupby="leiden_scVI",
    key="rank_genes_wilcoxon",
    var_names=markers_T_cd8,
    values_to_plot="logfoldchanges",
    cmap="bwr",
    vmin=-4,
    vmax=4,
    min_logfoldchange=1,
    colorbar_title="log fold change",
    save="general_dotplot_cd8.pdf",
    show=True,
)

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata,
    groupby="leiden_scVI",
    key="rank_genes_wilcoxon",
    var_names=markers_T_other,
    values_to_plot="logfoldchanges",
    cmap="bwr",
    vmin=-4,
    vmax=4,
    min_logfoldchange=1,
    colorbar_title="log fold change",
    save="general_dotplot_other.pdf",
    show=True,
)

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata,
    groupby="leiden_scVI",
    key="rank_genes_wilcoxon",
    var_names=markers_ILC,
    values_to_plot="logfoldchanges",
    cmap="bwr",
    vmin=-4,
    vmax=4,
    min_logfoldchange=1,
    colorbar_title="log fold change",
    save="general_dotplot_ILC.pdf",
    show=True,
)

In [ ]:
sc.pl.dotplot(
    adata,
    groupby="leiden_scVI",
    use_raw=True,
    var_names=markers_T_general,
    cmap="plasma_r",
    standard_scale="var",
    vmin=0,
    vmax=1,
    colorbar_title="Mean expression",
    dendrogram=False,
    save="TNKILC_dotplot_mean_general.pdf",
    show=True,
)

In [ ]:
sc.pl.dotplot(
    adata,
    groupby="leiden_scVI",
    use_raw=True,
    var_names=markers_T_cd4,
    cmap="plasma_r",
    standard_scale="var",
    vmin=0,
    vmax=1,
    colorbar_title="Mean expression",
    dendrogram=False,
    save="TNKILC_dotplot_mean_cd4.pdf",
    show=True,
)

In [ ]:
sc.pl.dotplot(
    adata,
    groupby="leiden_scVI",
    use_raw=True,
    var_names=markers_T_cd8,
    cmap="plasma_r",
    standard_scale="var",
    vmin=0,
    vmax=1,
    colorbar_title="Mean expression",
    dendrogram=False,
    save="TNKILC_dotplot_mean_cd8.pdf",
    show=True,
)

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata,
    groupby="leiden_scVI",
    key="rank_genes_wilcoxon",
    var_names=dissocation_markers_dict,
    values_to_plot="logfoldchanges",
    cmap="bwr",
    vmin=-4,
    vmax=4,
    min_logfoldchange=1,
    colorbar_title="log fold change",
    save="HSP_dotplot.pdf",
    show=True,
)

In [ ]:
pval_thresh = 0.05
log2fc_thresh = 1
cluster_de_genes = dict()
for cluster in sorted(set(adata.obs["leiden_scVI"])):
    cluster_de_genes[cluster] = sc.get.rank_genes_groups_df(
        adata,
        group=cluster,
        key="rank_genes_wilcoxon",
        pval_cutoff=pval_thresh,
        log2fc_min=log2fc_thresh,
        log2fc_max=None,
    ).sort_values("logfoldchanges", ascending=False)

# write to excel file DE genes per cluster
# Create a Pandas Excel writer using XlsxWriter as the engine.
path2save = DIR2SAVE.joinpath(
    "TNKILC_pval{}_log2fc{}.xlsx".format(pval_thresh, log2fc_thresh)
)

with pd.ExcelWriter(path2save) as writer:
    for cluster in list(cluster_de_genes.keys()):

        # get celltype of cluster
        # celltype = np.unique(adata[adata.obs['leiden']==cluster,:].obs['cell identity'])[0]

        cluster_de_genes[cluster].to_excel(
            writer, sheet_name="cluster{}".format(cluster)
        )

In [ ]:
sc.pl.umap(adata, color="Therapy")

In [ ]:
# write to file
adata.write(
    DIR2SAVE.joinpath(
        "Multiome_Che_Wu_CRC_LM_integrated_scvi_hvg_TNKILC_clustering.h5ad"
    )
)

In [ ]:
DIR2SAVE

In [ ]:
ax = sc.pl.umap(adata, size=10, show=False)
sc.pl.umap(
    adata[(adata.obs["cell_source"] == "Wu-Cells") & (adata.obs["leiden_scVI"] == "0")],
    size=10,
    color="Cell_subtype",
    ax=ax,
)

In [ ]:
adata[adata.obs["leiden_scVI"] == "2"].obs.Cell_subtype.value_counts()

In [7]:
adata = sc.read_h5ad(
    DIR2SAVE.joinpath(
        "Multiome_Che_Wu_CRC_LM_integrated_scvi_hvg_TNKILC_clustering.h5ad"
    )
)

In [ ]:
markers2plot = [
    "IL17A",
    "CCR4",
    "CCR6",
    "IL1R1",
    "IL6R",
    "IL21R",
    "IL23R",
    "TGFBR1",
    "RORA",
    "RORC",
    "BATF",
    "IRF4",
]

sc.pl.umap(
    adata,
    color=markers2plot,
    use_raw=True,
    vmin=0.0,
    vmax="p99",
    color_map="plasma_r",  #'RdPu',
    save="markers_th17.pdf",
    show=True,
)